In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt 
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime as dt
from secrets import secrets
from skill_api import extract_skills, extract_ignore

In [2]:
# Skills
df_skills = pd.read_csv('skills_db2/skill.csv')
SKILLS = df_skills['Skill'].unique().tolist()
# Redundant skills
df_redskills = pd.read_excel('skills/Other Skills.xlsx')
RED_SKILLS = df_redskills['Skill'].unique().tolist()
# Duplicate skills
df_dupskills = pd.read_excel('skills/Other Skills.xlsx', sheet_name='Duplicates')
DUP_SKILLS = df_dupskills.set_index('Skill').to_dict()['Parent']
SKILLS.extend(list(DUP_SKILLS.keys()))

In [4]:
query = """
select Job_ID, Job_Title, Company, Country, Date_Posted, Description, Keywords_Present, Title_Keywords 
from {}
where Date_Posted > '2020-04-01'
and Description != 'No Description'
"""

def extract_df(job):
    df = pd.read_sql(query.format(job), engine)
    df.drop_duplicates(inplace=True)
    no_skill = df['Keywords_Present'].isna()
    df['Num_Skill'] = 0
    df.loc[~no_skill, 'Num_Skill'] = df.loc[~no_skill, 'Keywords_Present'].apply(lambda x: len(x.split(',')))
    df = df.loc[df['Num_Skill'] <= 3]
    print('Number of rows', len(df))
    return df

def extract_skills_all(df):
    i = 0
    job_info_list = []
    initial = dt.now()
    interval = dt.now()
    print_every = int(len(df) / 20)
    for _, job in df.iterrows():
        i += 1
        if i % print_every == 0:
            print("{} jobs processed. Time taken: {}".format(i, dt.now() - interval))
            interval = dt.now()
        all_info = job['Job_Title'] + ' ' + job['Description']
        all_skills = extract_skills(all_info, SKILLS)
        keep_skills, _ = extract_ignore(all_skills, RED_SKILLS, DUP_SKILLS)
        keep_skills.sort()
        # ignore_skills.sort()
        job_info_list.append({
            'title': job['Job_Title'],
            'company': job['Company'],
            'country': job['Country'],
            'date_posted': job['Date_Posted'],
            'description': job['Description'],
            'indeed_skills': job['Keywords_Present'],
            'skills': keep_skills,
        })
    print("Total time taken: {}".format(dt.now() - initial))
    return pd.DataFrame.from_dict(job_info_list)

In [5]:
engine = create_engine(secrets['indeed_db'])
df_ds = extract_df('Data_Scientist')
df_ds.head()

Number of rows 2896


,Job_ID,Job_Title,Company,Country,Date_Posted,Description,Keywords_Present,Title_Keywords,Num_Skill
2,4b5160535b55a659,Junior- Mid level Data Scientist openings,ClientSolv Technologies,USA,2020-05-08,Company DescriptionClientSolv Technologies is ...,'C','Junior',1
3,d5326d3f749525a9,Data Scientist,Triplebyte,USA,2020-05-07,About TriplebyteWe are a platform that helps e...,"'C', 'Machine Learning', 'R'",None,3
7,f2a0194148860a86,"Data Scientist/Epidemiologist, Activity #2",Heluna Health,USA,2020-04-23,Project: LA County Novel Coronavirus (COVID-19...,"'C', 'R', 'SQL'",None,3
10,fcabe7775b900c1c,Senior Data Scientist,SAIC,USA,2020-05-08,Job DescriptionDescriptionSAIC is seeking a re...,"'C', 'R', 'Statistics'",'Senior',3
16,5e84b2024b6813c2,Data Scientist,Intrepid,USA,2020-04-10,Intrepid sets the standard for delivering exce...,"'C', 'Python', 'R'",None,3


In [6]:
engine.dispose()
df_dsj = extract_skills_all(df_ds)
df_dsj.head()

144 jobs processed. Time taken: 0:06:21.666809
288 jobs processed. Time taken: 0:06:03.697011
432 jobs processed. Time taken: 0:06:08.532298
576 jobs processed. Time taken: 0:05:37.191274
720 jobs processed. Time taken: 0:06:11.201109
864 jobs processed. Time taken: 0:06:10.987541
1008 jobs processed. Time taken: 0:06:39.284815
1152 jobs processed. Time taken: 0:07:07.009025
1296 jobs processed. Time taken: 0:06:38.922956
1440 jobs processed. Time taken: 0:07:19.292091
1584 jobs processed. Time taken: 0:08:23.135630
1728 jobs processed. Time taken: 0:06:35.178270
1872 jobs processed. Time taken: 0:09:03.286620
2016 jobs processed. Time taken: 0:06:46.331302
2160 jobs processed. Time taken: 0:06:09.596201
2304 jobs processed. Time taken: 0:05:39.781271
2448 jobs processed. Time taken: 0:06:32.337418
2592 jobs processed. Time taken: 0:05:15.511003
2736 jobs processed. Time taken: 0:06:29.782548
2880 jobs processed. Time taken: 0:07:09.106723
Total time taken: 2:13:09.571922


,title,company,country,date_posted,description,indeed_skills,skills
0,Junior- Mid level Data Scientist openings,ClientSolv Technologies,USA,2020-05-08,Company DescriptionClientSolv Technologies is ...,'C',"[Analytical, Business Administration, Hardware..."
1,Data Scientist,Triplebyte,USA,2020-05-07,About TriplebyteWe are a platform that helps e...,"'C', 'Machine Learning', 'R'","[Bayesian Inference, Data Acquisition, Dataset..."
2,"Data Scientist/Epidemiologist, Activity #2",Heluna Health,USA,2020-04-23,Project: LA County Novel Coronavirus (COVID-19...,"'C', 'R', 'SQL'","[Analysis, Data Management, Data Visualization..."
3,Senior Data Scientist,SAIC,USA,2020-05-08,Job DescriptionDescriptionSAIC is seeking a re...,"'C', 'R', 'Statistics'","[Agile, Analytical, Business Process, Dashboar..."
4,Data Scientist,Intrepid,USA,2020-04-10,Intrepid sets the standard for delivering exce...,"'C', 'Python', 'R'","[Administration, Administrative Law, Analytica..."


In [7]:
df_dsj.to_csv('indeed-insights/data_scientist_3.csv', index=False)


## K Means Clustering

In [10]:
skills = []

for _, row in df_dsj.iterrows():
    for s in row['skills']:
        if s not in skills:
            skills.append(s)

len(skills)

1516

In [52]:
import numpy as np

job_skills = df_dsj['skills'].tolist()
X = []

for job in job_skills:
    arr = np.zeros(len(skills))
    for s in job:
        arr[skills.index(s)] = 1
    X.append(arr)

X = np.array(X)
X.shape

(2896, 1516)

In [58]:
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

# clustering = DBSCAN(metric=dist_func).fit(X)
# clustering = DBSCAN().fit(X)
clustering = KMeans(n_clusters=5, random_state=42).fit(X)
clustering

KMeans(n_clusters=5, random_state=42)

In [59]:
clustering.labels_

array([4, 1, 4, ..., 0, 4, 2], dtype=int32)

In [60]:
df2 = df_dsj.copy()
df2['cluster'] = clustering.labels_
df2.head()

,title,company,country,date_posted,description,indeed_skills,skills,cluster
0,Junior- Mid level Data Scientist openings,ClientSolv Technologies,USA,2020-05-08,Company DescriptionClientSolv Technologies is ...,'C',"[Analytical, Business Administration, Hardware...",4
1,Data Scientist,Triplebyte,USA,2020-05-07,About TriplebyteWe are a platform that helps e...,"'C', 'Machine Learning', 'R'","[Bayesian Inference, Data Acquisition, Dataset...",1
2,"Data Scientist/Epidemiologist, Activity #2",Heluna Health,USA,2020-04-23,Project: LA County Novel Coronavirus (COVID-19...,"'C', 'R', 'SQL'","[Analysis, Data Management, Data Visualization...",4
3,Senior Data Scientist,SAIC,USA,2020-05-08,Job DescriptionDescriptionSAIC is seeking a re...,"'C', 'R', 'Statistics'","[Agile, Analytical, Business Process, Dashboar...",2
4,Data Scientist,Intrepid,USA,2020-04-10,Intrepid sets the standard for delivering exce...,"'C', 'Python', 'R'","[Administration, Administrative Law, Analytica...",2


In [61]:
df2['cluster'].unique()

array([4, 1, 2, 3, 0], dtype=int32)

In [75]:
def get_skill_count(df, cluster):
    df_temp = df.loc[df['cluster'] == cluster]
    skill_count = {}
    for _, row in df_temp.iterrows():
        for s in row['skills']:
            if s in skill_count:
                skill_count[s] += 1
            else:
                skill_count[s] = 1
    skill_count = {k: v for k, v in sorted(skill_count.items(), key=lambda item: item[1], reverse=True)}
    return skill_count

In [76]:
get_skill_count(df2, 0)

{'TestNG': 379,
 'Testing': 361,
 'Analytical': 139,
 'Training': 120,
 'Communication': 117,
 'Design': 116,
 'Computer Science': 98,
 'Problem Solving': 92,
 'Research': 88,
 'Documentation': 84,
 'Market': 76,
 'Collaboration': 75,
 'Platform': 73,
 'Electronics': 67,
 'Analysis': 66,
 'Innovation': 63,
 'Law': 62,
 'Data Analysis': 59,
 'Research and Development (R&D)': 58,
 'Artificial Intelligence (AI)': 58,
 'Data Science': 56,
 'Engineering': 55,
 'Reporting': 54,
 'Machine Learning': 49,
 'Leadership': 48,
 'Database': 48,
 'Sales': 48,
 'Security': 47,
 'Management': 46,
 'Protocol': 45,
 'Information Technology (IT)': 44,
 'Microsoft Access': 44,
 'Algorithm': 43,
 'Insurance': 43,
 'Project Management': 42,
 'Modelling': 42,
 'Network': 41,
 'Automation': 41,
 'Statistics': 40,
 'Sentry': 40,
 'Selection': 39,
 'Presentation': 38,
 'Visualization': 36,
 'Agile': 36,
 'Programming': 36,
 'Manufacturing': 35,
 'Writing': 35,
 'Mathematics': 34,
 'Software Engineering': 34,
 '

In [77]:
get_skill_count(df2, 1)

{'Machine Learning': 450,
 'Computer Science': 323,
 'Data Science': 295,
 'Artificial Intelligence (AI)': 251,
 'Analytical': 216,
 'Research': 213,
 'Engineering': 199,
 'Algorithm': 194,
 'Platform': 171,
 'Python': 169,
 'Design': 163,
 'Statistics': 159,
 'Leadership': 149,
 'Mathematics': 129,
 'Communication': 129,
 'Innovation': 128,
 'Collaboration': 127,
 'Deep Learning': 123,
 'R': 101,
 'Security': 100,
 'Analysis': 100,
 'Modelling': 89,
 'Programming': 88,
 'Software Engineering': 88,
 'Training': 84,
 'Data Analysis': 82,
 'Law': 78,
 'Problem Solving': 74,
 'Big Data': 72,
 'Data Analytics': 72,
 'Insurance': 68,
 'Presentation': 67,
 'Microsoft Access': 66,
 'Dataset': 64,
 'Play': 63,
 'Decision Making': 62,
 'Market': 62,
 'Predictive Model': 61,
 'Natural Language Processing (NLP)': 61,
 'Email': 60,
 'Information Technology (IT)': 59,
 'Healthcare': 58,
 'Computer Vision': 57,
 'Visualization': 57,
 'Data Mining': 54,
 'Programming Language': 53,
 'Database': 53,
 

In [78]:
get_skill_count(df2, 2)

{'Analytical': 282,
 'Statistics': 262,
 'Computer Science': 178,
 'Mathematics': 174,
 'Data Science': 152,
 'Reporting': 149,
 'Statistical Analysis': 144,
 'Training': 143,
 'Data Analysis': 137,
 'Research': 132,
 'Design': 130,
 'Leadership': 126,
 'Presentation': 122,
 'Visualization': 120,
 'Communication': 119,
 'Problem Solving': 118,
 'Innovation': 111,
 'Collaboration': 97,
 'Data Management': 95,
 'Engineering': 89,
 'Dataset': 87,
 'Processing': 86,
 'Documentation': 86,
 'R': 83,
 'Microsoft Access': 82,
 'Programming': 80,
 'Biostatistics': 80,
 'Management': 78,
 'Data Visualization': 77,
 'Law': 77,
 'Database': 75,
 'Data Mining': 75,
 'Table': 75,
 'Healthcare': 70,
 'Accuracy': 69,
 'Methodology': 69,
 'Machine Learning': 68,
 'Medicine': 67,
 'Security': 64,
 'Economics': 64,
 'Electronics': 64,
 'SAS': 64,
 'Predictive Model': 59,
 'Decision Making': 59,
 'Information Technology (IT)': 58,
 'Network': 56,
 'Interpreter': 52,
 'Writing': 52,
 'Market': 51,
 'Subjec

In [79]:
get_skill_count(df2, 3)

{'Analytical': 324,
 'Communication': 305,
 'Market': 234,
 'Leadership': 213,
 'Training': 192,
 'Analysis': 191,
 'Design': 191,
 'Presentation': 185,
 'Innovation': 170,
 'Sales': 164,
 'Collaboration': 160,
 'Research': 153,
 'Platform': 149,
 'Reporting': 133,
 'Problem Solving': 122,
 'Project Management': 115,
 'Law': 109,
 'Marketing': 103,
 'Network': 92,
 'Data Science': 92,
 'Management': 88,
 'Microsoft Excel': 80,
 'English': 77,
 'Life Science': 76,
 'Writing': 73,
 'Manufacturing': 73,
 'Research and Development (R&D)': 72,
 'Engineering': 72,
 'Insurance': 71,
 'Artificial Intelligence (AI)': 70,
 'Information Technology (IT)': 69,
 'Finance': 68,
 'Decision Making': 68,
 'Documentation': 68,
 'Budget': 63,
 'Written Communication': 61,
 'Translation': 59,
 'Consulting': 58,
 'Investment': 58,
 'Data Analytics': 55,
 'Data Analysis': 54,
 'Administration': 54,
 'Organizational Skill': 54,
 'Email': 53,
 'Agile': 53,
 'Accuracy': 50,
 'Microsoft Access': 50,
 'Processing

In [80]:
get_skill_count(df2, 4)

{'Research': 261,
 'Design': 202,
 'Analytical': 181,
 'Training': 170,
 'Communication': 139,
 'Analysis': 134,
 'Data Analysis': 122,
 'Data Science': 114,
 'Platform': 114,
 'Management': 114,
 'Computer Science': 113,
 'Artificial Intelligence (AI)': 110,
 'Engineering': 105,
 'Database': 100,
 'Leadership': 99,
 'Written Communication': 96,
 'Information Technology (IT)': 93,
 'Security': 92,
 'Innovation': 91,
 'Statistics': 88,
 'Reporting': 82,
 'Documentation': 81,
 'Email': 79,
 'Microsoft Access': 78,
 'Research and Development (R&D)': 75,
 'Python': 75,
 'Marketing': 74,
 'Writing': 74,
 'R': 73,
 'Market': 73,
 'Protocol': 72,
 'Presentation': 71,
 'English': 70,
 'Interpreter': 68,
 'Data Analytics': 67,
 'Administration': 66,
 'Collaboration': 66,
 'Machine Learning': 65,
 'Problem Solving': 65,
 'Product Development': 60,
 'Microsoft Excel': 59,
 'Structured Query Language (SQL)': 58,
 'Visualization': 57,
 'Law': 57,
 'Sales': 56,
 'Project Management': 52,
 'Molecular

## Topic Modeling

In [84]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')

doc_term_matrix = count_vect.fit_transform(df_dsj['description'].values.astype('U'))

In [85]:
doc_term_matrix

<2896x19160 sparse matrix of type '<class 'numpy.int64'>'
	with 557958 stored elements in Compressed Sparse Row format>

In [86]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [88]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 20 words for topic #{i+1}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print('\n')

Top 20 words for topic #1:
['years', 'process', 'related', 'opportunity', 'sales', 'analysis', 'technical', 'technology', 'new', 'ability', 'solutions', 'customer', 'status', 'skills', 'management', 'information', 'support', 'team', 'work', 'business']


Top 20 words for topic #2:
['related', 'working', 'regulatory', 'science', 'years', 'studies', 'analysis', 'team', 'lab', 'company', 'level', 'ensure', 'technical', 'new', 'support', 'development', 'work', 'scientific', 'laboratory', 'research']


Top 20 words for topic #3:
['company', 'statistical', 'develop', 'teams', 'engineering', 'technical', 'working', 'design', 'skills', 'new', 'analytics', 'years', 'machine', 'development', 'product', 'learning', 'science', 'business', 'work', 'team']


Top 20 words for topic #4:
['science', 'including', 'computer', 'level', 'time', 'job', 'related', 'program', 'provide', 'degree', 'application', 'requirements', 'analysis', 'statistical', 'work', 'information', 'required', 'education', 'researc

In [89]:
topic_values = LDA.transform(doc_term_matrix)
print(topic_values.shape)
topic_values[1]

(2896, 5)


array([9.63525147e-04, 9.60411707e-04, 9.96153116e-01, 9.61663809e-04,
       9.61283339e-04])

In [90]:
df2['topic'] = topic_values.argmax(axis=1)
df2.head()

,title,company,country,date_posted,description,indeed_skills,skills,cluster,topic
0,Junior- Mid level Data Scientist openings,ClientSolv Technologies,USA,2020-05-08,Company DescriptionClientSolv Technologies is ...,'C',"[Analytical, Business Administration, Hardware...",4,1
1,Data Scientist,Triplebyte,USA,2020-05-07,About TriplebyteWe are a platform that helps e...,"'C', 'Machine Learning', 'R'","[Bayesian Inference, Data Acquisition, Dataset...",1,2
2,"Data Scientist/Epidemiologist, Activity #2",Heluna Health,USA,2020-04-23,Project: LA County Novel Coronavirus (COVID-19...,"'C', 'R', 'SQL'","[Analysis, Data Management, Data Visualization...",4,3
3,Senior Data Scientist,SAIC,USA,2020-05-08,Job DescriptionDescriptionSAIC is seeking a re...,"'C', 'R', 'Statistics'","[Agile, Analytical, Business Process, Dashboar...",2,0
4,Data Scientist,Intrepid,USA,2020-04-10,Intrepid sets the standard for delivering exce...,"'C', 'Python', 'R'","[Administration, Administrative Law, Analytica...",2,0


## Managerial Roles

In [96]:
df3 = df_dsj.loc[df_dsj['title'].str.contains('manager', case=False)]
df3.head()

,title,company,country,date_posted,description,indeed_skills,skills
105,Principal Data Scientist Manager - Cloud Hardw...,Microsoft,USA,2020-05-02,To delight our customers in a Cloud First worl...,"'C', 'R'","[Applied Statistics, Automation, Cloud Service..."
136,Arity-Data Scientist-Sr Manager,Allstate,USA,2020-04-10,"Founded by The Allstate Corporation in 2016, A...","'C', 'Excel', 'R'","[Agile, Architecture, Business Analysis, Busin..."
202,Project Manager - Data Science | Chicago IL,Photon,USA,2020-04-28,Project Manager - Data Science | Chicago IL - ...,"'C', 'R', 'SQL'","[Apache Spark, Business Intelligence (BI), Dat..."
208,Corporate Communication Manager - Artificial I...,TuSimple,USA,2020-05-11,"This role is located in San Diego, CA. Relocat...","'Artificial Intelligence', 'C', 'R'","[Algorithm, Array, Artificial Intelligence (AI..."
234,Project Manager (Data Science & Public Health),ICF,USA,2020-04-29,ICF seeks a Project Manager with a data scienc...,"'C', 'Excel', 'R'","[Analytical, Business Analysis, Collaboration,..."


In [97]:
skill_count = {}
for _, row in df3.iterrows():
    for s in row['skills']:
        if s in skill_count:
            skill_count[s] += 1
        else:
            skill_count[s] = 1
skill_count = {k: v for k, v in sorted(skill_count.items(), key=lambda item: item[1], reverse=True)}
skill_count

{'Analytical': 120,
 'Leadership': 109,
 'Data Science': 91,
 'Communication': 87,
 'Computer Science': 86,
 'Design': 84,
 'Engineering': 79,
 'Platform': 75,
 'Project Management': 75,
 'Market': 75,
 'Innovation': 73,
 'Collaboration': 70,
 'Machine Learning': 67,
 'Research': 66,
 'Presentation': 64,
 'Training': 61,
 'Artificial Intelligence (AI)': 60,
 'Sales': 55,
 'Reporting': 54,
 'Marketing': 52,
 'Statistics': 46,
 'Problem Solving': 42,
 'Management': 41,
 'Documentation': 41,
 'Analysis': 40,
 'Law': 39,
 'TestNG': 38,
 'Mathematics': 36,
 'Architecture': 34,
 'Decision Making': 34,
 'Information Technology (IT)': 34,
 'Testing': 34,
 'Budget': 32,
 'Product Management': 32,
 'English': 31,
 'Written Communication': 30,
 'Finance': 29,
 'Database': 29,
 'Agile': 28,
 'Business Development': 28,
 'Business Intelligence (BI)': 27,
 'Consulting': 27,
 'R': 27,
 'Data Analytics': 25,
 'Microsoft Access': 25,
 'Algorithm': 24,
 'Email': 24,
 'Network': 24,
 'Product Development